In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time, json
import sys
sys.path.append("../")
sys.path.append("../chess_llm_interpretability")
import os
import torch
import numpy as np

import logging
import src.utils.logging_utils as logging_utils
import src.utils.env_utils as env_utils
import src.functional as functional
import src.models as models
import src.tokens as tokens

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")

2024-10-23 21:49:02 __main__ INFO     torch.__version__='2.5.0+cu124', torch.version.cuda='12.4'


In [3]:
# MODEL_KEY = "meta-llama/Llama-3.2-3B-Instruct"
# MODEL_KEY = "meta-llama/Llama-3.1-8B-Instruct"

# MODEL_KEY = "meta-llama/Llama-3.2-3B"
# MODEL_KEY = "google/gemma-2-2b"
# MODEL_KEY = "google/gemma-2-9b-it"
# MODEL_KEY = "meta-llama/Llama-3.1-8B"
MODEL_KEY = "meta-llama/Llama-3.2-3B"

mt = models.ModelandTokenizer(
    model_key=MODEL_KEY,
    torch_dtype=torch.bfloat16,
)

2024-10-23 21:49:02 src.models WARNING  meta-llama/Llama-3.1-8B-Instruct not found in models
If not found in cache, model will be downloaded from HuggingFace to cache directory


2024-10-23 21:49:04 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2024-10-23 21:50:37 src.models INFO     loaded model <meta-llama/Llama-3.1-8B-Instruct> | size: 15316.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [29]:
prompt = "The city of Paris is in the country of Italy."
inputs = tokens.prepare_input(prompt, mt)

In [9]:
from src.functional import patchscope, get_hs, logit_lens
from src.tokens import prepare_input, find_token_range

# prompt = "The city of Paris is in the country of Italy."
prompt = "Amazon's former CEO attended the Oscars."
inputs = prepare_input(prompt, mt, return_offset_mapping=True)

token_range = find_token_range(
    string = prompt,
    substring = "CEO",
    tokenizer=mt,
    offset_mapping=inputs["offset_mapping"][0],
)

mt.tokenizer.decode(inputs["input_ids"][0][token_range[0]:token_range[1]])  # CEO

32

In [30]:
h = functional.get_hs(
    mt = mt,
    input = inputs,
    locations = [(mt.layer_name_format.format(14), token_range[1] - 1)]
)

h.shape

torch.Size([4096])

In [11]:
interested_tokens = ["True", "False"]
mt.tokenizer(interested_tokens)

{'input_ids': [[128000, 2575], [128000, 4139]], 'attention_mask': [[1, 1], [1, 1]]}

In [44]:
target_prompt = '''<|start_header_id|>user<|end_header_id|>

True or false: placeholder placeholder placeholder placeholder placeholder<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

'''
functional.patchscope(
    mt = mt, 
    h = h,
    target_prompt = target_prompt,
    layer_idx = 3,
    interested_tokens = [2575, 4139],
    k = 5
)

([PredictedToken(token='**', prob=0.6400095224380493, logit=21.25, token_id=334),
  PredictedToken(token='False', prob=0.16181974112987518, logit=19.875, token_id=4139),
  PredictedToken(token='That', prob=0.04636213555932045, logit=18.625, token_id=4897),
  PredictedToken(token='A', prob=0.03610686957836151, logit=18.375, token_id=32),
  PredictedToken(token='I', prob=0.02189992181956768, logit=17.875, token_id=40)],
 {2575: (8,
   PredictedToken(token='True', prob=0.013282973319292068, logit=17.375, token_id=2575)),
  4139: (2,
   PredictedToken(token='False', prob=0.16181974112987518, logit=19.875, token_id=4139))})